In [1]:
import requests
import re
import xmltodict
from bs4 import BeautifulSoup

response = requests.get("https://boardgamegeek.com/browse/boardgame/page/1")
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find('table', {'class': 'collection_table'})
rows = table.find_all('tr')[1:]

board_games = []
for row in rows:
    columns = row.find_all('td')
    if len(columns) < 6:
        continue
    
    ## Get the information in the summary table
    rank = int(columns[0].text.strip())

    image_html_tag = str(columns[1].find("a").find("img"))
    id_html_tag = str(columns[1].find("a"))
    game_id = re.search('/(\d+)/', id_html_tag).group(1)
    image_url = re.search('src="(.*?)"', image_html_tag).group(1)

    name = columns[2].find_all('a')[0].text.strip()
    description = columns[2].find('p', class_='smallefont dull').text.strip()

    geek_rating = columns[3].text.strip()
    avg_rating = columns[4].text.strip()

    num_ratings = columns[5].text.strip()

    ## Call the api to get more detailed information about the game
    response = requests.get("https://www.boardgamegeek.com/xmlapi2/thing?id=224517&stats=1&comments=1")
    # Convert the XML response to a dictionary
    data_dict = xmltodict.parse(response.content)


    board_game = {'id':game_id, 'rank': rank, 'name': name, 'description': description, 'img_url':image_url}
    board_games.append(board_game)

In [71]:
len(board_games)

100

In [80]:
response = requests.get("https://boardgamegeek.com/browse/boardgame/")
soup = BeautifulSoup(response.content, 'html.parser')
num_pages = int(soup.find('a', title='last page').text.strip("[] "))
num_pages

1443

In [79]:
print(soup.find('a', title='last page').text.strip("[] "))

1443


In [71]:
response = requests.get("https://www.boardgamegeek.com/xmlapi2/thing?id=224517&comments=1")
# Convert the XML response to a dictionary
data_dict = xmltodict.parse(response.content)

In [72]:
data_dict['items']['item']['comments']['comment']

[{'@username': '1 Family Meeple',
  '@rating': 'N/A',
  '@value': 'SLEEVED[IMG]https://cf.geekdo-static.com/mbs/mb_5855_0.gif[/IMG] Deluxe Edition with Clay Coins'},
 {'@username': '13inha', '@rating': 'N/A', '@value': 'G'},
 {'@username': '1bez',
  '@rating': '10',
  '@value': 'Great game, full controllo of your strategy through constant adjustment of your tactic watching what your opponents do.'},
 {'@username': '1x0r', '@rating': 'N/A', '@value': "Haven't played yet"},
 {'@username': '2bit',
  '@rating': '7.5',
  '@value': "Very clever game, enjoyable overall.  Plus points: Great production values for all components and very good value for money. Lovely artwork everywhere. Smooth game play mechanisms - ironed out some oddities of the original game. Fortunes change frequently, so the winner could be anyone.  Minus points: The board and card artwork is generally too dark (even on the light side of the board), making it hard for some people to see detail clearly, especially under artif

In [57]:
# Get the total number of comments about this game, we should take a percentage of this by getting the next page until we have at least X percent of comments
num_comments = int(data_dict['items']['item']['comments']['@totalitems'])

In [58]:
num_comments

5239

In [31]:
game_id = '224517'

In [53]:
usernames = []
ratings = []
comments = []
page_num = 1
while len(ratings) < num_comments * 0.2:
    response = requests.get("https://boardgamegeek.com/xmlapi/boardgame/224517?comments=1&page={page_num}")
    print(response.content)
    page_num += 1
    # Convert the XML response to a dictionary
    data_dict = xmltodict.parse(response.content.decode())
    if 'boardgames' not in data_dict or 'boardgame' not in data_dict['boardgames']:
        break
    for comment in data_dict['boardgames']['boardgame']['comment']:
        if comment['@rating'] == 'N/A':
            continue
        usernames.append(comment["@username"])
        ratings.append(comment['@rating'])
        comments.append(comment['#text'])





b'<boardgames termsofuse="https://boardgamegeek.com/xmlapi/termsofuse">\n\t\t\t\t\t<boardgame objectid="224517">\n\t\t\t<yearpublished>2018</yearpublished>\n\t\t\t<minplayers>2</minplayers>\n\t\t\t<maxplayers>4</maxplayers>\n\t\t\t<playingtime>120</playingtime>\n\t\t\t<minplaytime>60</minplaytime>\n\t\t\t<maxplaytime>120</maxplaytime>\n\t\t\t<age>14</age>\n\n\t\t\t\t\t\t\t<name  sortindex="1">Brass. \xd0\x91\xd1\x96\xd1\x80\xd0\xbc\xd1\x96\xd0\xbd\xd0\xb3\xd0\xb5\xd0\xbc</name>\n\t\t\t\t\t\t\t<name  sortindex="1">Brass. \xd0\x91\xd0\xb8\xd1\x80\xd0\xbc\xd0\xb8\xd0\xbd\xd0\xb3\xd0\xb5\xd0\xbc</name>\n\t\t\t\t\t\t\t<name primary="true" sortindex="1">Brass: Birmingham</name>\n\t\t\t\t\t\t\t<name  sortindex="1">\xe3\x83\x96\xe3\x83\xa9\xe3\x82\xb9\xef\xbc\x9a\xe3\x83\x90\xe3\x83\xbc\xe3\x83\x9f\xe3\x83\xb3\xe3\x82\xac\xe3\x83\xa0</name>\n\t\t\t\t\t\t\t<name  sortindex="1">\xe5\xb7\xa5\xe6\xa5\xad\xe9\x9d\xa9\xe5\x91\xbd\xef\xbc\x9a\xe4\xbc\xaf\xe6\x98\x8e\xe7\xbf\xb0</name>\n\t\t\t\t\t\t\t

In [54]:
len(ratings)

1095

In [51]:
data_dict['boardgames']['boardgame']['comment']

[{'@username': '1 Family Meeple',
  '@rating': 'N/A',
  '#text': 'SLEEVED[IMG]https://cf.geekdo-static.com/mbs/mb_5855_0.gif[/IMG]\nDeluxe Edition with Clay Coins'},
 {'@username': '13inha', '@rating': 'N/A', '#text': 'G'},
 {'@username': '1bez',
  '@rating': '10',
  '#text': 'Great game, full controllo of your strategy through constant adjustment of your tactic watching what your opponents do.'},
 {'@username': '1x0r', '@rating': 'N/A', '#text': "Haven't played yet"},
 {'@username': '2bit',
  '@rating': '7.5',
  '#text': "Very clever game, enjoyable overall.\n\nPlus points:\nGreat production values for all components and very good value for money.\nLovely artwork everywhere.\nSmooth game play mechanisms - ironed out some oddities of the original game.\nFortunes change frequently, so the winner could be anyone.\n\nMinus points:\nThe board and card artwork is generally too dark (even on the light side of the board), making it hard for some people to see detail clearly, especially under 

In [139]:
int(data_dict['items']['item']['maxplayers']['@value'])

'4'

In [158]:
## Extract the best player counts for this game based on the BGG user poll
num_players_poll = [ poll for poll in data_dict['items']['item']['poll'] if poll['@name']=='suggested_numplayers'][0]
num_votes = int(num_players_poll['@totalvotes'])
num_players_dict = {}
for poll_result in num_players_poll['results']:    
    recommended = 0
    not_recommended = 0
    player_count = poll_result['@numplayers']
    for vote in poll_result['result']:
        match vote['@value']:
            case 'Best':
                recommended += int(vote['@numvotes'])
                not_recommended -= int(vote['@numvotes'])
            case 'Recommended':
                recommended += int(vote['@numvotes'])
            case 'Not Recommended':
                not_recommended += int(vote['@numvotes'])
    rec = recommended - not_recommended
    
    if '+' in player_count:
        max_players = int(data_dict['items']['item']['maxplayers']['@value'])
        for i in range(int(player_count.strip("+")), max_players + 1):
            if i in num_players_dict:
                num_players_dict[i] = num_players_dict[i] + rec
            else:
                num_players_dict[i] = rec
    else:
        num_players_dict[int(player_count)] = rec

num_players_dict = {(k, v/num_votes) for k,v in num_players_dict.items()}

In [159]:
num_players_dict

{(1, -0.46705202312138727),
 (2, 0.7236994219653179),
 (3, 1.3988439306358382),
 (4, 0.8520231213872832)}

In [126]:
data_dict['items']['item'].keys()

dict_keys(['@type', '@id', 'thumbnail', 'image', 'name', 'description', 'yearpublished', 'minplayers', 'maxplayers', 'poll', 'playingtime', 'minplaytime', 'maxplaytime', 'minage', 'link'])

In [187]:
data_dict['items']['item']['image']

'https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__original/img/FpyxH41Y6_ROoePAilPNEhXnzO8=/0x0/filters:format(jpeg)/pic3490053.jpg'

In [182]:
data_dict['items']['item']['playingtime']['@value']

'120'

In [185]:
data_dict['items']['item']['maxplaytime']['@value']

'120'

In [107]:
## Get the long (ish) description
data_dict['items']['item']['description']

"Brass: Birmingham is an economic strategy game sequel to Martin Wallace' 2007 masterpiece, Brass. Brass: Birmingham tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.&#10;&#10;As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands.&#10;&#10;Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):&#10;&#10;1) Build - Pay required resources and place an industry tile.&#10;2) Network - Add a rail / canal link, expanding your network.&#10;3) Develop - Increase the VP value of an industry.&#10;4) Sell - Sell your cotton, manufactured goods and pottery.&#10;5) Loan - Take a &pound;30 loan and reduce your income.&#10;&#10;Brass: Birmingham also features a new sixth action:&#10;&#10;6) Scout - Discard three cards and take a wild location

In [170]:
## Get the complexity score
data_dict['items']['item']['statistics']['ratings']['averageweight']['@value']

'3.8983'

In [179]:
categories = []
mechanics = []
publishers = []
artists = []
designers = []
other_implementations = []
families = []
for data in data_dict['items']['item']['link']:
    match data['@type']:
        case "boardgamecategory":
            categories.append(data['@value'])
        case "boardgamemechanic":
            mechanics.append(data['@value'])
        case 'boardgamepublisher':
            publishers.append(data['@value'])
        case 'boardgamedesigner':
            designers.append(data['@value'])
        case 'boardgameartist':
            artists.append(data['@value'])
        case 'boardgameimplementation':
            other_implementations.append(data['@id'])
        case 'boardgamefamily':
            families.append(data['@value'])
        case _:
            print(data['@type'])


boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgamefamily
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory
boardgameaccessory


In [118]:
[d['@totalvotes'] for d in data_dict['items']['item']['poll']]

['865', '126', '43']

In [2]:
html = b'<!DOCTYPE html>\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\n<head>\n\t<meta charset=\'utf-8\'>\n\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t<meta content=\'yes\' name=\'apple-mobile-web-app-capable\'>\n\t<meta content=\'IE=edge,chrome=1\' http-equiv=\'X-UA-Compatible\'>\n\n\t\t\t<title>Brass: Birmingham | Board Game | BoardGameGeek</title>\n\t\n\t\t<link rel="canonical" href="https://boardgamegeek.com/boardgame/224517/brass-birmingham" />\n\n<link rel="apple-touch-icon" \thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\n<link rel="shortcut icon" \t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="icon" \t\t\t\t\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\n<link rel="search" \t\t\t\thref="/game-opensearch.xml" type="application/opensearchdescription+xml" title="BGG Game Search" />\n<meta name="apple-mobile-web-app-title" content="BGG">\n\n<meta name="theme-color" content="#2e2b47">\n<link rel="preconnect" href="https://api.geekdo.com" />\n\n<link rel="preload" as="image" href="https://cf.geekdo-images.com/e2e0wdO8oVNzD0ZAhzi6MQ__itemheader/img/Zj1GBUN2m-z4zX2-QWn31fiKEE4=/800x450/filters:quality(30):strip_icc()/pic3509697.jpg" media="(min-width: 768px)">\n<link rel="preload" as="image" href="https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__itemrep/img/giNUMut4HAl-zWyQkGG0YchmuLI=/fit-in/246x300/filters:strip_icc()/pic3490053.jpg">\n\n\t<meta name="title" content="Brass: Birmingham">\n\n\t<meta property="og:image" content="https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__opengraph/img/9JP25fFDv5TUeo3h0i2ky6xXwQk=/0x249:1200x879/fit-in/1200x630/filters:strip_icc()/pic3490053.jpg" />\n\t<meta property="og:title" content="Brass: Birmingham" />\n\t<meta property="og:site_name" content="BoardGameGeek" />\n\t<meta property="og:url" content="https://boardgamegeek.com/boardgame/224517/brass-birmingham" />\n\t<meta property="og:type" content="website" />\n\t<meta property="og:description" content="Build networks, grow industries, and navigate the world of the Industrial Revolution." />\n\n\t<meta name="twitter:card" content="summary_large_image" />\n\t<meta name="twitter:title" content="Brass: Birmingham" />\n\t<meta name="twitter:image:src" content="https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__opengraph/img/9JP25fFDv5TUeo3h0i2ky6xXwQk=/0x249:1200x879/fit-in/1200x630/filters:strip_icc()/pic3490053.jpg" />\n\t<meta name="twitter:description" content="Build networks, grow industries, and navigate the world of the Industrial Revolution." />\n\n\t<meta name="description" content="Build networks, grow industries, and navigate the world of the Industrial Revolution.">\n\n\t<meta name="keywords" content="board game, boardgames, boardgame, board, games, game, hobby, boardgamegeek, geek, geekdo">\n\n<script>window.AdSlots = window.AdSlots || {\n\tcmd: [],\n\tdisableScripts: [\'gpt\'],\n\trenderOnFirstLoad: false,\n\tdivCheck: false\n};</script>\n\n\n\t\t\t\t\t\t<link rel=\'stylesheet\' type=\'text/css\' href=\'https://cf.geekdo-static.com/static/geekui_master2_6447431534f64.css\'>\n\t\t\t\n\t<base href="/">\n\n\t<script src="https://kit.fontawesome.com/42d3f5a072.js" crossorigin="anonymous"></script>\n\n<script>\n\tvar GEEK = GEEK || {};\n\tGEEK.adBlock = [];\n\tGEEK.adConfig = {"blockleaderboard":false,"blockskyscraper":false,"noadsense":false};\n\tGEEK.adSlots = {"dfp-leaderboard":{"name":"boardgame_leaderboard_728x90"},"dfp-skyscraper":{"name":"boardgame_skyscraper_160x600"},"dfp-medrect":{"name":"boardgame_rectangle_300x250"},"dfp-repeater":{"name":"boardgame_home_repeater"},"dfp-medrect-reserved-home":{"name":"boardgame_reserved_home_300x250"},"dfp-leaderboard-lg":{"name":"boardgame_home_hero"},"dfp-home-sidekick":{"name":"boardgame_home_repeater"},"dfp-inline-post":{"name":"boardgame_inline_post"}};\n\tGEEK.legacyAds = [];\n\tGEEK.googleTargets = {"gameid":["224517"],"companyid":["21765","3475","25074","21608","11043","34522","31071","8832","8820","4785","42147","36210","46500","30677","36186","7466","41423"],"temp_co":["hot","hot"],"familyid":["61830","61842","61831","55549","55559","55718","55547","64690","64689","64680","64681","64679","73364","58912","74286","56191","65041","66892","66899","66898","17519","65191","14759","8374","22135","77349","70948","26397","57499","50152","78198","4705","5497","66445"],"personid":["32887","70571","70568","38179","32943","6"],"propertyid":["1021","2040","2902","1088","2904","2900","2081","2710","3100","2849","1034","1011","2826","2897"],"temp_ppl":["hot"],"temp_game":["hot"]};\n\tGEEK.userid = 0;\n\tGEEK.domainname = \'boardgamegeek.com\';\n\tGEEK.domain = \'boardgame\';\n\tGEEK.geekitemPreload = {"item":{"itemdata":[{"datatype":"geekitem_fielddata","fieldname":"name","title":"Primary Name","primaryname":true,"required":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"name"},{"datatype":"geekitem_fielddata","fieldname":"alternatename","title":"Alternate Names","alternate":true,"unclickable":true,"fullcredits":true,"subtype":"boardgame","keyname":"alternatename"},{"datatype":"geekitem_fielddata","fieldname":"yearpublished","title":"Year Released","fullcredits":true,"subtype":"boardgame","keyname":"yearpublished"},{"datatype":"geekitem_fielddata","fieldname":"minplayers","title":"Minimum Players","subtype":"boardgame","keyname":"minplayers"},{"datatype":"geekitem_fielddata","fieldname":"maxplayers","title":"Maximum Players","subtype":"boardgame","keyname":"maxplayers"},{"datatype":"geekitem_fielddata","fieldname":"minplaytime","title":"Minimum Playing Time","createposttext":" minutes","posttext":" minutes","subtype":"boardgame","keyname":"minplaytime"},{"datatype":"geekitem_fielddata","fieldname":"maxplaytime","title":"Maximum Playing Time","createposttext":" minutes","posttext":" minutes","subtype":"boardgame","keyname":"maxplaytime"},{"datatype":"geekitem_fielddata","fieldname":"minage","title":"Mfg Suggested Ages","createtitle":"Minimum Age","posttext":" and up","subtype":"boardgame","keyname":"minage"},{"datatype":"geekitem_fielddata","fieldname":"override_rankable","title":"Override Rankable","table":"geekitem_items","options":[{"value":1,"title":"yes"},{"value":0,"title":"no"}],"adminonly":true,"subtype":"boardgame","keyname":"override_rankable"},{"datatype":"geekitem_fielddata","fieldname":"targetco_url","unclickable":true,"title":"Target Co Order Link","subtype":"boardgame","keyname":"targetco_url"},{"datatype":"geekitem_fielddata","fieldname":"shopify_store_url","unclickable":true,"title":"Shopify Store Link","subtype":"boardgame","keyname":"shopify_store_url"},{"datatype":"geekitem_fielddata","fieldname":"walmart_id","unclickable":true,"title":"Walmart Item Id","nullable":true,"subtype":"boardgame","keyname":"walmart_id"},{"datatype":"geekitem_fielddata","fieldname":"instructional_videoid","unclickable":true,"title":"Promoted Instructional Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"instructional_videoid"},{"datatype":"geekitem_fielddata","fieldname":"summary_videoid","unclickable":true,"title":"Promoted Summary Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"summary_videoid"},{"datatype":"geekitem_fielddata","fieldname":"playthrough_videoid","unclickable":true,"title":"Promoted Playthrough Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"playthrough_videoid"},{"datatype":"geekitem_fielddata","fieldname":"focus_videoid","unclickable":true,"title":"Promoted In Focus Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"focus_videoid"},{"datatype":"geekitem_fielddata","fieldname":"howtoplay_videoid","unclickable":true,"title":"Promoted BGG How to Play Video ID","validatemethod":"ValidateVideoid","nullable":true,"subtype":"boardgame","keyname":"howtoplay_videoid"},{"datatype":"geekitem_fielddata","fieldname":"bggstore_product","unclickable":true,"title":"Promoted BGG Store Product Name","nullable":true,"subtype":"boardgame","keyname":"bggstore_product"},{"datatype":"geekitem_fielddata","fieldname":"short_description","title":"Short Description","table":"geekitem_items","maxlength":85,"editfieldsize":60,"required":true,"subtype":"boardgame","keyname":"short_description"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamedesigner","linktype":"boardgamedesigner","self_prefix":"src","title":"Designer","titlepl":"Designers","fullcredits":true,"keyname":"boardgamedesigner"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamesolodesigner","linktype":"boardgamesolodesigner","self_prefix":"src","title":"Solo Designer","titlepl":"Solo Designers","fullcredits":true,"keyname":"boardgamesolodesigner"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgameartist","linktype":"boardgameartist","self_prefix":"src","title":"Artist","titlepl":"Artists","fullcredits":true,"keyname":"boardgameartist"},{"datatype":"geekitem_linkdata","other_objecttype":"company","other_subtype":"boardgamepublisher","linktype":"boardgamepublisher","self_prefix":"src","title":"Publisher","titlepl":"Publishers","required":true,"fullcredits":true,"keyname":"boardgamepublisher"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamedeveloper","linktype":"boardgamedeveloper","self_prefix":"src","title":"Developer","titlepl":"Developers","fullcredits":true,"keyname":"boardgamedeveloper"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamegraphicdesigner","linktype":"boardgamegraphicdesigner","self_prefix":"src","title":"Graphic Designer","titlepl":"Graphic Designers","fullcredits":true,"keyname":"boardgamegraphicdesigner"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamesculptor","linktype":"boardgamesculptor","self_prefix":"src","title":"Sculptor","titlepl":"Sculptors","fullcredits":true,"keyname":"boardgamesculptor"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgameeditor","linktype":"boardgameeditor","self_prefix":"src","title":"Editor","titlepl":"Editors","fullcredits":true,"keyname":"boardgameeditor"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamewriter","linktype":"boardgamewriter","self_prefix":"src","title":"Writer","titlepl":"Writers","fullcredits":true,"keyname":"boardgamewriter"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgameinsertdesigner","linktype":"boardgameinsertdesigner","self_prefix":"src","title":"Insert Designer","titlepl":"Insert Designers","fullcredits":true,"keyname":"boardgameinsertdesigner"},{"datatype":"geekitem_linkdata","other_objecttype":"family","other_subtype":"boardgamehonor","lookup_subtype":"boardgamehonor","linktype":"boardgamehonor","self_prefix":"src","title":"Honors","keyname":"boardgamehonor"},{"datatype":"geekitem_linkdata","other_objecttype":"property","other_subtype":"boardgamecategory","lookup_subtype":"boardgamecategory","linktype":"boardgamecategory","self_prefix":"src","title":"Category","titlepl":"Categories","showall_ctrl":true,"fullcredits":true,"overview_count":5,"keyname":"boardgamecategory"},{"datatype":"geekitem_linkdata","other_objecttype":"property","other_subtype":"boardgamemechanic","lookup_subtype":"boardgamemechanic","linktype":"boardgamemechanic","self_prefix":"src","title":"Mechanism","titlepl":"Mechanisms","showall_ctrl":true,"fullcredits":true,"overview_count":5,"keyname":"boardgamemechanic"},{"datatype":"geekitem_linkdata","lookup_subtype":"boardgame","other_objecttype":"thing","other_subtype":"boardgameexpansion","linktype":"boardgameexpansion","self_prefix":"src","title":"Expansion","keyname":"boardgameexpansion"},{"datatype":"geekitem_linkdata","other_objecttype":"version","other_subtype":"boardgameversion","linktype":"boardgameversion","other_is_dependent":true,"required":true,"loadlinks":true,"self_prefix":"src","title":"Version","createtitle":"Versions","uneditable":true,"keyname":"boardgameversion"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgameexpansion","self_prefix":"dst","title":"Expands","overview_count":100,"keyname":"expandsboardgame"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgameintegration","correctioncomment":"Only for stand-alone games that integrate with other stand-alone games. <b>NOT<\\/b> for expansions.","self_prefix":"src","title":"Integrates With","overview_count":4,"keyname":"boardgameintegration"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgamecompilation","self_prefix":"dst","correctioncomment":"Items contained in this item (if this is a compilation, for example)","title":"Contains","overview_count":4,"keyname":"contains"},{"datatype":"geekitem_linkdata","lookup_subtype":"boardgame","other_objecttype":"thing","other_subtype":"boardgame","linktype":"boardgamecompilation","self_prefix":"src","title":"Contained in","keyname":"containedin"},{"datatype":"geekitem_linkdata","lookup_subtype":"boardgame","other_objecttype":"thing","other_subtype":"boardgame","linktype":"boardgameimplementation","self_prefix":"src","correctioncomment":"Add the \\"child\\" item(s) that reimplement this game","title":"Reimplemented By","overview_count":4,"keyname":"reimplementation"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgame","lookup_subtype":"boardgame","linktype":"boardgameimplementation","self_prefix":"dst","correctioncomment":"Add the \\"parent\\" item(s) for this game, if it reimplements a previous game","title":"Reimplements","overview_count":4,"keyname":"reimplements"},{"datatype":"geekitem_linkdata","other_objecttype":"family","other_subtype":"boardgamefamily","lookup_subtype":"boardgamefamily","linktype":"boardgamefamily","self_prefix":"src","fullcredits":true,"title":"Family","overview_count":10,"keyname":"boardgamefamily"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"videogamebg","lookup_subtype":"videogame","linktype":"videogamebg","self_prefix":"src","adminonly":true,"title":"Video Game Adaptation","titlepl":"Video Game Adaptations","keyname":"videogamebg"},{"datatype":"geekitem_linkdata","other_objecttype":"family","other_subtype":"boardgamesubdomain","lookup_subtype":"boardgamesubdomain","linktype":"boardgamesubdomain","polltype":"boardgamesubdomain","display_inline":true,"self_prefix":"src","title":"Type","showall_ctrl":true,"uneditable":true,"createposttext":"Enter the subdomain for this item.","keyname":"boardgamesubdomain"},{"datatype":"geekitem_linkdata","other_objecttype":"thing","other_subtype":"boardgameaccessory","lookup_subtype":"boardgameaccessory","linktype":"boardgameaccessory","self_prefix":"src","title":"Accessory","titlepl":"Accessories","addnew":true,"keyname":"boardgameaccessory"},{"datatype":"geekitem_linkdata","linktype":"commerceweblink","other_objecttype":"weblink","other_subtype":"commerceweblink","self_prefix":"src","title":"Commerce Weblinks","adminaccess":["admgr"],"keyname":"commerceweblink"},{"datatype":"geekitem_polldata","title":"User Suggested # of Players","polltype":"numplayers","keyname":"userplayers"},{"datatype":"geekitem_polldata","title":"User Suggested Ages","polltype":"playerage","keyname":"playerage"},{"datatype":"geekitem_polldata","title":"Language Dependence","polltype":"languagedependence","keyname":"languagedependence"},{"datatype":"geekitem_polldata","title":"Subdomain","polltype":"boardgamesubdomain","keyname":"subdomain"},{"datatype":"geekitem_polldata","title":"Weight","polltype":"boardgameweight","keyname":"boardgameweight"}],"relatedlinktypes":["boardgameaccessory","contains","containedin","reimplementation","reimplements","videogamebg","boardgameintegration"],"classification_types":["boardgamesubdomain","boardgamecategory","boardgamemechanic","boardgamefamily","boardgameintegration","reimplementation","reimplements","contains","expandsboardgame"],"linkedforum_types":[{"title":"Board Game Forums","linkedforum_index":"boardgames","linkdata_index":"expandsboardgame","required_subtype":null},{"title":"Expansion Forums","linkedforum_index":"boardgameexpansions","linkdata_index":"boardgameexpansion","required_subtype":null},{"title":"Compilation Item Forums","linkedforum_index":"compilationitems","linkdata_index":"contains","required_subtype":"boardgamecompilation"},{"title":"Parent Item Forums","linkedforum_index":"reimplements","linkdata_index":"reimplements","required_subtype":"boardgameimplementation"},{"title":"Reimplementation Forums","linkedforum_index":"reimplementedby","linkdata_index":"reimplementation","required_subtype":null},{"title":"Integrates With","linkedforum_index":"boardgameintegration","linkdata_index":"boardgameintegration","required_subtype":"boardgameintegration"},{"title":"Video Game Forums","linkedforum_index":"videogamebgs","linkdata_index":"videogamebg","required_subtype":null}],"subtypename":"Board Game","honor_subtype":"boardgamehonor","honor_linktype":"boardgamehonor","credit_subtypes":[{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgamedesigner","linktype":"boardgamedesigner","self_prefix":"src","title":"Designer","titlepl":"Designers","fullcredits":true,"keyname":"boardgamedesigner"},{"datatype":"geekitem_linkdata","other_objecttype":"person","other_subtype":"boardgameartist","linktype":"boardgameartist","self_prefix":"src","title":"Artist","titlepl":"Artists","fullcredits":true,"keyname":"boardgameartist"},{"datatype":"geekitem_linkdata","other_objecttype":"company","other_subtype":"boardgamepublisher","linktype":"boardgamepublisher","self_prefix":"src","title":"Publisher","titlepl":"Publishers","required":true,"createsubtext":"(Can use Unknown, Looking for a publisher, Self-published, and Web-published - for Self and Web add URL in Notes to Admin box)","error":"Publisher Required","fullcredits":true,"keyname":"boardgamepublisher"}],"show_geekbuddy_analysis":true,"rankinfo":[{"prettyname":"Board Game Rank","shortprettyname":"Overall Rank","veryshortprettyname":"Overall","subdomain":null,"rankobjecttype":"subtype","rankobjectid":1,"rank":"1","baverage":"8.42561"},{"prettyname":"Strategy Game Rank","shortprettyname":"Strategy Rank","veryshortprettyname":"Strategy ","subdomain":"strategygames","rankobjecttype":"family","rankobjectid":5497,"rank":"1","baverage":"8.44993"}],"polls":{"userplayers":{"best":[{"min":3,"max":4}],"recommended":[{"min":2,"max":4}],"totalvotes":"867"},"playerage":"14+","languagedependence":"No necessary in-game text","subdomain":"","boardgameweight":{"averageweight":3.8978102189781021,"votes":"1781"}},"stats":{"usersrated":"37652","average":"8.616","baverage":"8.42561","stddev":"1.4209","avgweight":"3.8978","numweights":"1781","numgeeklists":"2522","numtrading":"162","numwanting":"1689","numwish":"15211","numowned":"52139","numprevowned":"1972","numcomments":"5236","numwishlistcomments":"1114","numhasparts":"17","numwantparts":"5","views":"3263600","playmonth":"2023-04","numplays":"95876","numplays_month":"1639","numfans":3480},"relatedcounts":{"news":5,"blogs":393,"weblink":28,"podcast":48},"itemid":1747810,"objecttype":"thing","objectid":224517,"label":"Board Game","labelpl":"Board Games","shortlabel":"Game","shortlabelpl":"Games","type":"things","id":"224517","href":"\\/boardgame\\/224517\\/brass-birmingham","subtype":"boardgame","subtypes":["boardgame","boardgameimplementation"],"versioninfo":{"gamepageorderurl":null,"shopifyitem":null},"name":"Brass: Birmingham","alternatename":null,"yearpublished":"2018","minplayers":"2","maxplayers":"4","minplaytime":"60","maxplaytime":"120","minage":"14","override_rankable":0,"targetco_url":"","shopify_store_url":null,"walmart_id":null,"instructional_videoid":"186574","summary_videoid":"187803","playthrough_videoid":null,"focus_videoid":null,"howtoplay_videoid":null,"bggstore_product":null,"short_description":"Build networks, grow industries, and navigate the world of the Industrial Revolution.","links":{"boardgamedesigner":[{"name":"Gavan Brown","objecttype":"person","objectid":"32887","primarylink":0,"itemstate":"approved","href":"\\/boardgamedesigner\\/32887\\/gavan-brown"},{"name":"Matt Tolman","objecttype":"person","objectid":"32943","primarylink":0,"itemstate":"approved","href":"\\/boardgamedesigner\\/32943\\/matt-tolman"},{"name":"Martin Wallace","objecttype":"person","objectid":"6","primarylink":0,"itemstate":"approved","href":"\\/boardgamedesigner\\/6\\/martin-wallace"}],"boardgamesolodesigner":[],"boardgameartist":[{"name":"Lina Cossette","objecttype":"person","objectid":"70571","primarylink":0,"itemstate":"approved","href":"\\/boardgameartist\\/70571\\/lina-cossette"},{"name":"David Forest","objecttype":"person","objectid":"70568","primarylink":0,"itemstate":"approved","href":"\\/boardgameartist\\/70568\\/david-forest"},{"name":"Damien Mammoliti","objecttype":"person","objectid":"38179","primarylink":0,"itemstate":"approved","href":"\\/boardgameartist\\/38179\\/damien-mammoliti"}],"boardgamepublisher":[{"name":"Roxley","objecttype":"company","objectid":"21765","primarylink":1,"itemstate":"approved","href":"\\/boardgamepublisher\\/21765\\/roxley"},{"name":"Arclight Games","objecttype":"company","objectid":"3475","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/3475\\/arclight-games"},{"name":"BoardM Factory","objecttype":"company","objectid":"25074","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/25074\\/boardm-factory"},{"name":"CMON Global Limited","objecttype":"company","objectid":"21608","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/21608\\/cmon-global-limited"},{"name":"Conclave Editora","objecttype":"company","objectid":"11043","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/11043\\/conclave-editora"},{"name":"CrowD Games","objecttype":"company","objectid":"34522","primarylink":0,"itemstate":"approved","href":"\\/boardgamepublisher\\/34522\\/crowd-games"}],"boardgamedeveloper":[],"boardgamegraphicdesigner":[],"boardgamesculptor":[],"boardgameeditor":[],"boardgamewriter":[],"boardgameinsertdesigner":[],"boardgamehonor":[{"name":"2018 Board Game Quest Awards Best Production Values Nominee","objecttype":"family","objectid":"61830","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/61830\\/2018-board-game-quest-awards-best-production-value"},{"name":"2018 Board Game Quest Awards Best Production Values Winner","objecttype":"family","objectid":"61842","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/61842\\/2018-board-game-quest-awards-best-production-value"},{"name":"2018 Board Game Quest Awards Best Strategy\\/Euro Game Nominee","objecttype":"family","objectid":"61831","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/61831\\/2018-board-game-quest-awards-best-strategyeuro-gam"},{"name":"2018 Golden Geek Best Board Game Artwork & Presentation Nominee","objecttype":"family","objectid":"55549","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/55549\\/2018-golden-geek-best-board-game-artwork-presentat"},{"name":"2018 Golden Geek Best Strategy Board Game Nominee","objecttype":"family","objectid":"55559","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/55559\\/2018-golden-geek-best-strategy-board-game-nominee"},{"name":"2018 Golden Geek Best Strategy Board Game Winner","objecttype":"family","objectid":"55718","primarylink":0,"itemstate":"approved","href":"\\/boardgamehonor\\/55718\\/2018-golden-geek-best-strategy-board-game-winner"}],"boardgamecategory":[{"name":"Economic","objecttype":"property","objectid":"1021","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1021\\/economic"},{"name":"Industry \\/ Manufacturing","objecttype":"property","objectid":"1088","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1088\\/industry-manufacturing"},{"name":"Post-Napoleonic","objecttype":"property","objectid":"2710","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/2710\\/post-napoleonic"},{"name":"Trains","objecttype":"property","objectid":"1034","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1034\\/trains"},{"name":"Transportation","objecttype":"property","objectid":"1011","primarylink":0,"itemstate":"approved","href":"\\/boardgamecategory\\/1011\\/transportation"}],"boardgamemechanic":[{"name":"Hand Management","objecttype":"property","objectid":"2040","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2040\\/hand-management"},{"name":"Income","objecttype":"property","objectid":"2902","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2902\\/income"},{"name":"Loans","objecttype":"property","objectid":"2904","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2904\\/loans"},{"name":"Market","objecttype":"property","objectid":"2900","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2900\\/market"},{"name":"Network and Route Building","objecttype":"property","objectid":"2081","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/2081\\/network-and-route-building"},{"name":"Tags","objecttype":"property","objectid":"3100","primarylink":0,"itemstate":"approved","href":"\\/boardgamemechanic\\/3100\\/tags"}],"boardgameexpansion":[],"boardgameversion":[{"name":"Chinese edition","objecttype":"version","objectid":"523778","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/523778\\/chinese-edition"},{"name":"Czech\\/English edition","objecttype":"version","objectid":"558467","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/558467\\/czechenglish-edition"},{"name":"English deluxe edition","objecttype":"version","objectid":"417443","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/417443\\/english-deluxe-edition"},{"name":"English retail edition","objecttype":"version","objectid":"351052","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/351052\\/english-retail-edition"},{"name":"French edition","objecttype":"version","objectid":"435701","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/435701\\/french-edition"},{"name":"German first edition","objecttype":"version","objectid":"491771","primarylink":0,"itemstate":"approved","href":"\\/boardgameversion\\/491771\\/german-first-edition"}],"expandsboardgame":[],"boardgameintegration":[],"contains":[],"containedin":[],"reimplementation":[],"reimplements":[{"name":"Brass: Lancashire","objecttype":"thing","objectid":"28720","primarylink":0,"itemstate":"approved","href":"\\/boardgame\\/28720\\/brass-lancashire"}],"boardgamefamily":[{"name":"Cities: Birmingham (England)","objecttype":"family","objectid":"17519","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/17519\\/cities-birmingham-england"},{"name":"Components: Multi-Use Cards","objecttype":"family","objectid":"65191","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/65191\\/components-multi-use-cards"},{"name":"Country: England","objecttype":"family","objectid":"14759","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/14759\\/country-england"},{"name":"Crowdfunding: Kickstarter","objecttype":"family","objectid":"8374","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/8374\\/crowdfunding-kickstarter"},{"name":"Crowdfunding: Spieleschmiede","objecttype":"family","objectid":"22135","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/22135\\/crowdfunding-spieleschmiede"},{"name":"Digital Implementations: Steam","objecttype":"family","objectid":"77349","primarylink":0,"itemstate":"approved","href":"\\/boardgamefamily\\/77349\\/digital-implementations-steam"}],"videogamebg":[{"name":"Brass: Birmingham","objecttype":"thing","objectid":"356820","primarylink":0,"itemstate":"approved","href":"\\/videogame\\/356820\\/brass-birmingham"}],"boardgamesubdomain":[{"name":"Strategy Games","objecttype":"family","objectid":"5497","primarylink":0,"itemstate":"approved","href":"\\/boardgamesubdomain\\/5497\\/strategy-games"}],"boardgameaccessory":[{"name":"Brass: Birmingham \\/ Lancashire \\u2013 reDrewno Insert","objecttype":"thing","objectid":"334571","primarylink":1,"itemstate":"approved","href":"\\/boardgameaccessory\\/334571\\/brass-birmingham-lancashire-redrewno-insert"},{"name":"Brass Birmingham: Spike Craft Insert","objecttype":"thing","objectid":"373838","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/373838\\/brass-birmingham-spike-craft-insert"},{"name":"Brass: Birmingham \\/ Lancashire \\u2013 Eurohell Design Insert","objecttype":"thing","objectid":"347851","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/347851\\/brass-birmingham-lancashire-eurohell-design-insert"},{"name":"Brass: Birmingham \\/ Lancashire \\u2013 Folded Space Insert","objecttype":"thing","objectid":"306354","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/306354\\/brass-birmingham-lancashire-folded-space-insert"},{"name":"Brass: Birmingham \\u2013 Dicetroyers Organizer","objecttype":"thing","objectid":"350761","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/350761\\/brass-birmingham-dicetroyers-organizer"},{"name":"Brass: Birmingham \\u2013 e-Raptor Insert","objecttype":"thing","objectid":"309493","primarylink":0,"itemstate":"approved","href":"\\/boardgameaccessory\\/309493\\/brass-birmingham-e-raptor-insert"}],"commerceweblink":[{"name":"Geek Game Shop listing for Brass Birmingham","objecttype":"weblink","objectid":"323285","primarylink":0,"itemstate":"approved","href":"\\/commerceweblink\\/323285\\/geek-game-shop-listing-brass-birmingham"}]},"linkcounts":{"boardgamedesigner":3,"boardgamesolodesigner":0,"boardgameartist":3,"boardgamepublisher":17,"boardgamedeveloper":0,"boardgamegraphicdesigner":0,"boardgamesculptor":0,"boardgameeditor":0,"boardgamewriter":0,"boardgameinsertdesigner":0,"boardgamehonor":20,"boardgamecategory":5,"boardgamemechanic":9,"boardgameexpansion":0,"boardgameversion":19,"expandsboardgame":0,"boardgameintegration":0,"contains":0,"containedin":0,"reimplementation":0,"reimplements":1,"boardgamefamily":13,"videogamebg":1,"boardgamesubdomain":1,"boardgameaccessory":14,"commerceweblink":1},"commercelinks":[{"orderstartdate":"0000-00-00","orderenddate":"0000-00-00","ordertype":"geekgameshop","commerceurl":"https:\\/\\/geekgameshop.com\\/products\\/brass-birmingham","rect_ad_url":"","seller_name":"","premium":"","shopifyitem":null}],"secondarynamescount":0,"alternatenamescount":5,"primaryname":{"nameid":"1272946","name":"Brass: Birmingham","sortindex":"1","primaryname":"1","translit":"Brass: Birmingham"},"description":"<p><em><strong>Brass: Birmingham<\\/strong><\\/em> is an economic strategy game sequel to Martin Wallace\' 2007 masterpiece, <em>Brass<\\/em>. <em>Brass: Birmingham<\\/em> tells the story of competing entrepreneurs in Birmingham during the industrial revolution, between the years of 1770-1870.<\\/p>\\n\\n<p>As in its predecessor, you must develop, build, and establish your industries and network, in an effort to exploit low or high market demands.<\\/p>\\n\\n<p>Each round, players take turns according to the turn order track, receiving two actions to perform any of the following actions (found in the original game):<\\/p>\\n\\n<p>1) <strong>Build<\\/strong> - Pay required resources and place an industry tile.<br \\/>\\n2) <strong>Network<\\/strong> - Add a rail \\/ canal link, expanding your network.<br \\/>\\n3) <strong>Develop<\\/strong> - Increase the VP value of an industry.<br \\/>\\n4) <strong>Sell<\\/strong> - Sell your cotton, manufactured goods and pottery.<br \\/>\\n5) <strong>Loan<\\/strong> - Take a &pound;30 loan and reduce your income.<\\/p>\\n\\n<p><em>Brass: Birmingham<\\/em> also features a new sixth action:<\\/p>\\n\\n<p>6) <strong>Scout<\\/strong> - Discard three cards and take a wild location and wild industry card. (This action replaces Double Action Build in original <em>Brass<\\/em>.)<\\/p>\\n\\n<p>The game is played over two halves: the canal era (years 1770-1830) and the rail era (years 1830-1870). To win the game, score the most VPs. VPs are counted at the end of each half for the canals, rails and established (flipped) industry tiles.<\\/p>\\n\\n<p><em>Birmingham<\\/em> features dynamic scoring canals\\/rails. Instead of each flipped industry tile giving a static 1 VP to all connected canals and rails, many industries give 0 or even 2 VPs. This provides players with the opportunity to score much higher value canals in the first era, and creates interesting strategy with industry placement.<\\/p>\\n\\n<p>Iron, coal, and cotton are three industries which appear in both the original <em>Brass<\\/em> as well as in <em>Brass: Birmingham<\\/em>.<\\/p>\\n\\n<p><strong>New &quot;Sell&quot; system<\\/strong><\\/p>\\n\\n<p>Brewing has become a fundamental part of the culture in Birmingham. You must now sell your product through traders located around the edges of the board. Each of these traders is looking for a specific type of good each game. To sell cotton, pottery, or manufactured goods to these traders, you must also &quot;grease the wheels of industry&quot; by consuming beer. For example, a level 1 cotton mill requires one beer to flip. As an incentive to sell early, the first player to sell to a trader receives free beer.<\\/p>\\n\\n<p><em>Birmingham<\\/em> features three all-new industry types:<\\/p>\\n\\n<p><strong>Brewery<\\/strong> - Produces precious beer barrels required to sell goods.<\\/p>\\n\\n<p><strong>Manufactured goods<\\/strong> - Function like cotton, but features eight levels. Each level of manufactured goods provides unique rewards, rather than just escalating in VPs, making it a more versatile (yet potentially more difficult) path vs cotton.<\\/p>\\n\\n<p><strong>Pottery<\\/strong> - These behemoths of Birmingham offer huge VPs, but at a huge cost and need to plan.<\\/p>\\n\\n<p><strong>Increased Coal and Iron Market size<\\/strong> - The price of coal and iron can now go up to &pound;8 per cube, and it\'s not uncommon.<\\/p>\\n\\n<p><em>Brass: Birmingham<\\/em> is a sequel to Brass. It offers a very different story arc and experience from its predecessor.<\\/p>\\n\\n","wiki":"<strong>Components:<\\/strong><br \\/>\\n<ul>\\n    <li> 1 Game board (double sided - light\\/dark)<\\/li>\\n    <li> 4 Player Mats (double sided - light\\/dark)<\\/li>\\n    <li> 4 Character Tiles (double sided - male\\/female)<\\/li>\\n    <li> 56 Link tiles (14 per colour)<\\/li>\\n    <li> 76 Cards [63,5x88 mm]<ul>\\n        <li> 8 Wild Cards (4 Location and 4 Industry)<\\/li>\\n        <li> 64 Location and Industry cards<\\/li>\\n        <li> 4 Player Aids<\\/li>\\n    <\\/ul><\\/li>\\n    <li> 4 VP markers (1 per colour)<\\/li>\\n    <li> 4 Income markers (1 per colour)<\\/li>\\n    <li> 30 Coal cubes (8mm)<\\/li>\\n    <li> 18 Iron cubes (8mm)<\\/li>\\n    <li> 15 Beer barrels<\\/li>\\n    <li> 67 Money tokens (Deluxe edition: 70 poker token)<\\/li>\\n    <li> 180 Industry tiles (45 per colour)<ul>\\n        <li> 11 Manufacturers<\\/li>\\n        <li> 11 Cotton Mills<\\/li>\\n        <li> 7 Breweries<\\/li>\\n        <li> 5 Potteries<\\/li>\\n        <li> 4 Iron Works<\\/li>\\n        <li> 7 Coal Mines<\\/li>\\n    <\\/ul><\\/li>\\n    <li> 9 Merchant Tiles<\\/li>\\n<\\/ul>\\n\\n<p><a class=\'anchor-target\' name=\\"Microbadges\\"><\\/a>\\t\\t\\t\\t<a class=\\"anchor-target\\" id=\\"toc0\\"><\\/a>\\n\\t\\t\\t\\t<div>\\n\\t\\t\\t\\t\\t<h3>Microbadges<\\/h3><\\/div><\\/p>\\n\\n<p><a href=\\"\\/microbadge\\/41582\\"><img alt=\'Microbadge: Brass: Birmingham fan\' onmouseover=\\"return overlib(\'Brass: Birmingham fan\', WRAP );\\" onmouseout=\\"return nd();\\" title=\'\'src=\\"data:image\\/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\\"onload=\\"lzld(this)\\"onerror=\\"lzld(this)\\"class=\'mbimg\'data-frz-src=\\"https:\\/\\/cf.geekdo-static.com\\/mbs\\/mb_41582_0.png\\"><\\/a> - Brass: Birmingham fan<\\/p>\\n\\n<p><a href=\\"\\/microbadge\\/41477\\"><img alt=\'Microbadge: Brass fan\' onmouseover=\\"return overlib(\'Brass fan\', WRAP );\\" onmouseout=\\"return nd();\\" title=\'\'src=\\"data:image\\/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==\\"onload=\\"lzld(this)\\"onerror=\\"lzld(this)\\"class=\'mbimg\'data-frz-src=\\"https:\\/\\/cf.geekdo-static.com\\/mbs\\/mb_41477_4.png\\"><\\/a> - Brass Fan<\\/p>\\n\\n<\\/div><\\/div>","website":{"url":"https:\\/\\/roxley.com\\/products\\/brass-birmingham-deluxe-edition","title":"Brass: Birmingham - Official Roxley page"},"imageid":"3490053","images":{"thumb":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__thumb\\/img\\/o18rjEemoWaVru9Y2TyPwuIaRfE=\\/fit-in\\/200x150\\/filters:strip_icc()\\/pic3490053.jpg","micro":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__micro\\/img\\/4Od3GYCiqptga0VbmyumPbJlBsU=\\/fit-in\\/64x64\\/filters:strip_icc()\\/pic3490053.jpg","square":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__square\\/img\\/DKV_8eUyd_ctCHiPv7QEydRUOVQ=\\/75x75\\/filters:strip_icc()\\/pic3490053.jpg","squarefit":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__squarefit\\/img\\/4lfOVn9A9ZCoZErBsDEiOEGWP8c=\\/fit-in\\/75x75\\/filters:strip_icc()\\/pic3490053.jpg","tallthumb":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__tallthumb\\/img\\/ybMA6Pa9t8etvDppu2iKEeJZkt4=\\/fit-in\\/75x125\\/filters:strip_icc()\\/pic3490053.jpg","previewthumb":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__previewthumb\\/img\\/Y9apAdg80y0-RV1Oxze6ZwzZe18=\\/fit-in\\/300x320\\/filters:strip_icc()\\/pic3490053.jpg","square200":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__square200\\/img\\/dNf_NYkJhQ6h2K0zWxhzRmJfAg8=\\/200x200\\/filters:strip_icc()\\/pic3490053.jpg"},"imagepagehref":"\\/image\\/3490053\\/brass-birmingham","imageurl":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__itemrep\\/img\\/giNUMut4HAl-zWyQkGG0YchmuLI=\\/fit-in\\/246x300\\/filters:strip_icc()\\/pic3490053.jpg","topimageurl":"https:\\/\\/cf.geekdo-images.com\\/e2e0wdO8oVNzD0ZAhzi6MQ__itemheader\\/img\\/Zj1GBUN2m-z4zX2-QWn31fiKEE4=\\/800x450\\/filters:quality(30):strip_icc()\\/pic3509697.jpg","imageSets":{"square100":{"src":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__square100\\/img\\/9I8MlquCKrLTqm-w32_K85PDU8M=\\/100x100\\/filters:strip_icc()\\/pic3490053.jpg","src@2x":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__square100@2x\\/img\\/dNf_NYkJhQ6h2K0zWxhzRmJfAg8=\\/200x200\\/filters:strip_icc()\\/pic3490053.jpg"},"mediacard":{"src":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__mediacard\\/img\\/jRj9wl9q5_zv1Og0fi8vLqYsXGE=\\/0x226:1200x901\\/288x162\\/filters:strip_icc()\\/pic3490053.jpg","src@2x":"https:\\/\\/cf.geekdo-images.com\\/x3zxjr-Vw5iU4yDPg70Jgw__mediacard@2x\\/img\\/3pW54SpsB2HIGKMbnvVH3GPN4NE=\\/0x226:1200x901\\/576x324\\/filters:strip_icc()\\/pic3490053.jpg"}},"itemstate":"approved","promoted_ad":null,"special_user":null,"instructional_video":{"id":"186574","type":"videos","videoid":186574,"title":"Brass: Birmingham - How To Play, by Watch It Play","description":"Want to learn how to play Brass: Birmingham? This video will teach you how!\\n\\n[b]Be sure to check the description of the video for a fully clickable index to specific instructions you might be interested in.[\\/b]\\n\\nIf you want to see any of our past videos, you can easily check out our playlists: https:\\/\\/www.youtube.com\\/user\\/WatchItPlayed\\/playlists?view=1&flow=grid\\n\\n[b]Consider subscribing to our channel so that you will know when new episodes are released. Find our channel here: http:\\/\\/www.youtube.com\\/WatchItPlayed[\\/b]\\n\\n*****************\\n\\nGet A Watch It Played Microbadge: http:\\/\\/www.boardgamegeek.com\\/microbadge\\/25384\\n\\nEmail: watchitplayed(at)live(dot)com\\nTwitter: @watchitplayed\\nPeriscope: @watchitplayed\\nFacebook: https:\\/\\/www.facebook.com\\/WatchItPlayed\\/\\nInstagram: @watchitplayed\\n\\nWe truly appreciate your views, feedback and subscriptions.","descriptionXml":"<?xml version=\\"1.0\\"?>\\n<xml><safehtml>Want to learn how to play Brass: Birmingham? This video will teach you how!&lt;br\\/&gt;&lt;br\\/&gt;&lt;strong&gt;Be sure to check the description of the video for a fully clickable index to specific instructions you might be interested in.&lt;\\/strong&gt;&lt;br\\/&gt;&lt;br\\/&gt;If you want to see any of our past videos, you can easily check out our playlists: &lt;a rel=\\"nofollow noreferrer noopener\\" target=\\"_blank\\" style=\\"word-break: break-word;\\" href=\\"https:\\/\\/www.youtube.com\\/user\\/WatchItPlayed\\/playlists?view=1&amp;flow=grid\\"&gt;https:\\/\\/www.youtube.com\\/user\\/WatchItPlayed\\/playlists?view=1&amp;amp;...&lt;\\/a&gt;&lt;br\\/&gt;&lt;br\\/&gt;&lt;strong&gt;Consider subscribing to our channel so that you will know when new episodes are released. Find our channel here: &lt;a rel=\\"nofollow noreferrer noopener\\" target=\\"_blank\\" style=\\"word-break: break-word;\\" href=\\"http:\\/\\/www.youtube.com\\/WatchItPlayed\\"&gt;http:\\/\\/www.youtube.com\\/WatchItPlayed&lt;\\/a&gt;&lt;\\/strong&gt;&lt;br\\/&gt;&lt;br\\/&gt;*****************&lt;br\\/&gt;&lt;br\\/&gt;Get A Watch It Played Microbadge: &lt;a rel=\\"nofollow noreferrer noopener\\" target=\\"_blank\\" style=\\"word-break: break-word;\\" href=\\"http:\\/\\/www.boardgamegeek.com\\/microbadge\\/25384\\"&gt;http:\\/\\/www.boardgamegeek.com\\/microbadge\\/25384&lt;\\/a&gt;&lt;br\\/&gt;&lt;br\\/&gt;Email: watchitplayed(at)live(dot)com&lt;br\\/&gt;Twitter: @watchitplayed&lt;br\\/&gt;Periscope: @watchitplayed&lt;br\\/&gt;Facebook: &lt;a rel=\\"nofollow noreferrer noopener\\" target=\\"_blank\\" style=\\"word-break: break-word;\\" href=\\"https:\\/\\/www.facebook.com\\/WatchItPlayed\\/\\"&gt;https:\\/\\/www.facebook.com\\/WatchItPlayed\\/&lt;\\/a&gt;&lt;br\\/&gt;Instagram: @watchitplayed&lt;br\\/&gt;&lt;br\\/&gt;We truly appreciate your views, feedback and subscriptions.<\\/safehtml><\\/xml>\\n","postdate":"2018-08-15T13:15:31+00:00","gallery":"instructional","submitter":247565,"hidden":false,"blocks_ads":false,"language":{"name":"English","id":"eng","type":"languages"},"source":{"type":"things","id":"224517"},"featured":false,"featuredPlacementSponsored":false,"video":{"host":"youtube","id":"Lzxf4tfq9as"},"href":"\\/video\\/186574\\/brass-birmingham\\/brass-birmingham-how-play-watch-it-play","canonical_link":"https:\\/\\/boardgamegeek.com\\/video\\/186574\\/brass-birmingham\\/brass-birmingham-how-play-watch-it-play","browse_href":"\\/boardgame\\/224517\\/brass-birmingham\\/videos\\/all","links":[{"rel":"self","uri":"\\/api\\/videos\\/186574"},{"rel":"thumbs.total","uri":"\\/api\\/videos\\/186574\\/reactions?totalonly=1"},{"rel":"thumbs","uri":"\\/api\\/videos\\/186574\\/reactions"},{"rel":"tips","uri":"\\/api\\/videos\\/186574\\/tips"},{"rel":"tips.total","uri":"\\/api\\/videos\\/186574\\/tips?totalonly=1"},{"rel":"comments","uri":"\\/api\\/videos\\/186574\\/comments"}]},"summary_video":{"id":"187803","type":"videos","videoid":187803,"title":"Brass: Birmingham \\u2014 game overview at Gen Con 2018","description":"","descriptionXml":"<?xml version=\\"1.0\\"?>\\n<xml\\/>\\n","postdate":"2018-08-25T14:05:42+00:00","gallery":"interview","submitter":15734,"hidden":false,"blocks_ads":false,"language":{"name":"English","id":"eng","type":"languages"},"source":{"type":"things","id":"224517"},"featured":false,"featuredPlacementSponsored":false,"video":{"host":"youtube","id":"l9g7Ez6M6a4"},"href":"\\/video\\/187803\\/brass-birmingham\\/brass-birmingham-game-overview-gen-con-2018","canonical_link":"https:\\/\\/boardgamegeek.com\\/video\\/187803\\/brass-birmingham\\/brass-birmingham-game-overview-gen-con-2018","browse_href":"\\/boardgame\\/224517\\/brass-birmingham\\/videos\\/all","links":[{"rel":"self","uri":"\\/api\\/videos\\/187803"},{"rel":"thumbs.total","uri":"\\/api\\/videos\\/187803\\/reactions?totalonly=1"},{"rel":"thumbs","uri":"\\/api\\/videos\\/187803\\/reactions"},{"rel":"tips","uri":"\\/api\\/videos\\/187803\\/tips"},{"rel":"tips.total","uri":"\\/api\\/videos\\/187803\\/tips?totalonly=1"},{"rel":"comments","uri":"\\/api\\/videos\\/187803\\/comments"}]}},"media":{"files":{"numitems":"114"},"images":{"numitems":824},"videos":{"numitems":223}},"videogalleries":{"galleries":[{"type":"all","name":"All"},{"type":"review","name":"Review"},{"type":"session","name":"Session"},{"type":"instructional","name":"Instructional"},{"type":"interview","name":"Interview"},{"type":"unboxing","name":"Unboxing"},{"type":"humor","name":"Humor"},{"type":"other","name":"Other"}]}};\n\tGEEK.geekitemSettings = {"fluidlayout":false,"absolutedates":false,"fulldescriptions":false,"overview_nohot":false,"recentitems_showcount":5,"modules":{"boardgameperson":[{"name":"description","enabled":true}],"boardgame":[{"name":"promotedvideos","enabled":true},{"name":"media","enabled":true},{"name":"description","enabled":true},{"name":"awards","enabled":true},{"name":"marketplace","adblock":"blockbuyacopy","enabled":true},{"name":"videos","enabled":true},{"name":"reviews","enabled":true},{"name":"recs","enabled":true},{"name":"forums","enabled":true},{"name":"files","enabled":true},{"name":"wiki","enabled":true},{"name":"moregame","enabled":true}]}};\n\tGEEK.geekitemModules = {"media":{"name":"media","html":"<media-module><\\/media-module>","prettyname":"Images"},"promotedvideos":{"name":"promotedvideos","html":"<promotedvideos-module geekitem=\\"overviewctrl.geekitem\\"><\\/promotedvideos-module>","prettyname":"Promoted Videos"},"classification":{"name":"classification","html":"<classifications-module geekitem=\\"overviewctrl.geekitem\\"><\\/classifications-module>","prettyname":"Classification"},"description":{"name":"description","html":"<description-module wikitext=\\"overviewctrl.geekitem.data.item.description\\" geekitem=\\"overviewctrl.geekitem\\"><\\/description-module>","prettyname":"Description"},"marketplace":{"name":"marketplace","html":"<marketplace-module geekitem=\\"overviewctrl.geekitem\\"><\\/marketplace-module>","prettyname":"Marketplace"},"videos":{"name":"videos","html":"<span class=\'hidden-xs\'>\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t<videos-overview-module ng-if=\'!overviewctrl.geekitemSettings.overview_nohot\'><\\/videos-overview-module>\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t\\t<videos-overview-module-recent ng-if=\'overviewctrl.geekitemSettings.overview_nohot\'><\\/videos-overview-module-recent>\\n\\t\\t\\t\\t\\t\\t\\t\\t\\t <\\/span>","prettyname":"Videos"},"reviews":{"name":"reviews","html":"<reviews-module disablelocation=\\"true\\"><\\/reviews-module>","prettyname":"Reviews"},"forums":{"name":"forums","html":"<forums-overview-module disablelocation=\\"true\\"><\\/forums-overview-module>","prettyname":"Forums"},"recs":{"name":"recs","html":"<recs-module-overview><\\/recs-module-overview>","prettyname":"Recommended Games"},"files":{"name":"files","html":"<files-overview-module disablelocation=\\"true\\"><\\/files-overview-module>","prettyname":"Files"},"wiki":{"name":"wiki","html":"<wiki-module wikitext=\\"overviewctrl.geekitem.data.item.wiki\\"><\\/wiki-module>","prettyname":"Wiki"},"moregame":{"name":"moregame","html":"<more-game-module geekitem=\\"overviewctrl.geekitem\\"><\\/more-game-module>","prettyname":"More of this Game"}};\n\tGEEK.geekGlobalSettings = {"shutdown_file_ops":"1","shutdown_storeimage_ops":"0","shutdown_edit_avatar":"0","shutdown_file_upload":"0","shutdown_file_download":"0","shutdown_image_upload":"0"};\n\tGEEK.geekimageSettings = {"galleries":{"all":{"name":"all","prettyname":"All","description":""},"game":{"name":"game","prettyname":"Game","description":" Images where the focus is on the unaltered game, box, and\\/or components."},"people":{"name":"people","prettyname":"People","description":"Images where people are prominently featured with the game."},"creative":{"name":"creative","prettyname":"Creative","description":"Images of the game that contain creative elements such as modified components (including custom painted miniatures), fun situations, storage solutions, screen shots of online implementations, advertisements, prototypes, or artwork. Any images where the game has been added to or altered."}}};\n\tGEEK.legacy = 0;\n\tGEEK.apiurlsPrefix = \'https://api.geekdo.com\';\n\n\t\n\t\t\n\t\tGEEK.apiurls = {\n\t\t\'root\': \'/api\',\n\t\t\'amazon\': \'/api/amazon\',\n\t\t\'files\': \'/api/files\',\n\t\t\'geekitems\': \'/api/geekitems\',\n\t\t\'images\': \'/api/images\',\n\t\t\'threads\': \'/api/forums/threads\',\n\t \t\'forums\': \'/api/forums\',\n\t\t\'videos\': \'/api/videos\',\n\t\t\'hotness\': \'/api/hotness\',\n\t\t\'dynamicinfo\': \'/api/dynamicinfo\',\n\t\t\'subtypeinfo\': \'/api/subtypeinfo\',\n\t\t\'geekbay\': \'/api/geekbay\',\n\t\t\'geekmarket\': \'/market/api/v1\',\n\t    \'geekmarketapi\': \'/api/market\',\n\t\t\'geeklists\': \'/api/geeklists\',\n\t\t\'reviews\': \'/api/forumreviews\',\n\t\t\'collections\': \'/api/collections\',\n\t\t\'linkeditems\': \'/api/geekitem/linkeditems\',\n\t\t\'subscriptions\': \'/api/subscriptions\',\n\t    \'fans\': \'/api/fans\',\n\t    \'geekpreviews\': \'/api/geekpreviews\',\n\t    \'geekpreviewitems\': \'/api/geekpreviewitems\',\n\t   \t\'geekpreviewparentitems\': \'/api/geekpreviewparentitems\',\n\t\t\'recs\': \'/api/geekitem/recs\',\n\t    \'awards\': \'/api/geekawards\',\n\t\t\'historicalrankgraph\':  \'/api/historicalrankgraph\',\n\t\t\'blueprint_recipes\': \'/api/blueprints/recipes\',\n\t\t\'affiliateads\' : \'/api/affiliateads\'\n\t};\n</script>\n\n<script async src="https://www.googletagmanager.com/gtag/js?id=UA-104725-1"></script>\n<script>\n\twindow.dataLayer = window.dataLayer || [];\n\tfunction gtag(){dataLayer.push(arguments);}\n\tgtag(\'js\', new Date());\n\tgtag(\'consent\', \'default\', {\n\t\t\'ad_storage\': \'denied\',\n\t\t\'analytics_storage\': \'denied\',\n\t\t\'personalization_storage\': \'denied\',\n\t\t\'wait_for_update\': 3000\n\t\t });\n\tgtag(\'config\', \'UA-104725-1\', {\n\t\t\'cookie_domain\': \'boardgamegeek.com\',\n\t\t\'send_page_view\': false\n\t});\n</script>\n\n    \t\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekangular17_master2_64474320f3809.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekui_master2_6447431534f64.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekitem_master2_6447431524aee.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekcollection_master2_64474320ed84c.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekoutputtemplates_master2_64474321a44d3.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekitemtemplates_master2_64474321acb42.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekcollectiontemplates_master2_64474321d7cff.js\'></script>\n\t\t\t\t\t<script type=\'text/javascript\' src=\'https://cf.geekdo-static.com/static/geekuicommontemplates_master2_64474321aa8cd.js\'></script>\n\t\t\t<script>\n\twindow.geekCookieConsent.gtagReady.then(function() {\n\t\tgtag(\'event\', \'page_view\');\n\t});\n</script>\n\t\n</head>\n\n<body ng-controller="GeekOutput_LayoutCtrl as layoutctrl"\n\t\tclass="domain-boardgame"\n\n\t\tng-class="{ \'has-no-max-width\' : layoutctrl.geekitemSettings.fluidlayout }">\n\n\n<div class=\'d-flex flex-column min-vh-100\'>\n        \t\n<div id="global-header-outer" class=\'global-header-outer\'\n\t  ng-controller="NavCtrl as navctrl"\n\t  click-out="navctrl.closeMobileMenu()">\n\n\t<geeknav-menu></geeknav-menu>\n</div>\n\n\t \n\t\n\t<main class=\'global-body flex-grow-1\'\n\t\t  id="mainbody"\n\t\t  ng-class="{ \'has-overlay-sidebar\': layoutctrl.showOverlaySidebar, \'has-hidden-fixed-sidebar\': layoutctrl.localStorage.hideFixedSidebar }">\n\t\t\n\t\t<a id=\'mainbodytarget\' tabindex="-1"></a>\n\n\t\t<!-- Home King Ad -->\n\t\t\n\t\t<div hide-ad-block="blockleaderboard" class="item-hero-layout">\n\t\t\t<div class="advertisement-leaderboard">\n\t\t\t\t<div class=\'center-block\' ng-dfp-ad="dfp-leaderboard"></div>\n\t\t\t</div>\n\t\t</div>\n\n\t\t<div class=\'global-body-content-container container-fluid\'>\n\t\t\t\t\t\t\t<geekoutput-sidebar deactivate-overlay-sidebar=\'layoutctrl.deactivateOverlaySidebar\' show-overlay-sidebar=\'layoutctrl.showOverlaySidebar\'></geekoutput-sidebar>\n         \n\t\t\t\n\t\t\t<div class=\'global-body-content pending\'  ng-class="{\'ready\': layoutctrl.ready}">\n\t\t\n\t\t\t\t<a id=\'maincontenttarget\' tabindex="-1"></a>\n\t\t\t\n\t\t\t\t \t\t\t\t\t\t\t\t<script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/8.2.2/highstock.src.min.js"></script>\n<script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/8.2.2/modules/accessibility.min.js"></script>\n<script src="https://cdnjs.cloudflare.com/ajax/libs/highcharts/8.2.2/themes/sand-signika.min.js"></script>\n\n<geekitem linkshowcount="6" subtype="boardgame"></geekitem>\n\n\n\n\t\t\t\t<div class=\'global-body-content-secondary\'>\n\t\t\t\t\t\t\t\t\t</div>\n\t\t\t</div>\n\t\t</div>\n\n\t</main>\n\t\t\t<geekoutput-footer></geekoutput-footer>\n\t</div>\n\n\n<script type="application/ld+json">\n{\n    "@context": "http://schema.org",\n    "@type": "Product",\n    "aggregateRating": {\n        "@type": "AggregateRating",\n        "ratingValue": "8.616",\n        "reviewCount": "37652",\n        "bestRating": "10",\n        "worstRating": "1"\n    },\n    "image": "https://cf.geekdo-images.com/x3zxjr-Vw5iU4yDPg70Jgw__itemrep/img/giNUMut4HAl-zWyQkGG0YchmuLI=/fit-in/246x300/filters:strip_icc()/pic3490053.jpg",\n    "name": "Brass: Birmingham",\n    "description": "Build networks, grow industries, and navigate the world of the Industrial Revolution."\n}\n</script>\n\n</body>\n</html>\n'

In [9]:
str(html)

'b\'<!DOCTYPE html>\\n<html ng-app="GeekApp" lang="en-US" ng-cloak>\\n<head>\\n\\t<meta charset=\\\'utf-8\\\'>\\n\\t<meta id="vp" name="viewport" content="width=device-width, initial-scale=1.0">\\n\\t\\t<meta content=\\\'yes\\\' name=\\\'apple-mobile-web-app-capable\\\'>\\n\\t<meta content=\\\'IE=edge,chrome=1\\\' http-equiv=\\\'X-UA-Compatible\\\'>\\n\\n\\t\\t\\t<title>Brass: Birmingham | Board Game | BoardGameGeek</title>\\n\\t\\n\\t\\t<link rel="canonical" href="https://boardgamegeek.com/boardgame/224517/brass-birmingham" />\\n\\n<link rel="apple-touch-icon" \\thref="https://cf.geekdo-static.com/icons/touch-icon180.png" />\\n<link rel="shortcut icon" \\t\\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\\n<link rel="icon" \\t\\t\\t\\t\\thref="https://cf.geekdo-static.com/icons/favicon2.ico" type="image/ico" />\\n<link rel="search" \\t\\t\\t\\thref="/game-opensearch.xml" type="application/opensearchdescription+xml" title="BGG Game Search" />\\n<meta name="a

In [158]:
response = requests.get("https://boardgamegeek.com/xmlapi/collection/Aarontu?rated=1")
data_dict = xmltodict.parse( response.content)

In [159]:
data_dict

{'items': {'@totalitems': '419',
  '@termsofuse': 'https://boardgamegeek.com/xmlapi/termsofuse',
  '@pubdate': 'Wed, 26 Apr 2023 06:29:58 +0000',
  'item': [{'@objecttype': 'thing',
    '@objectid': '5867',
    '@subtype': 'boardgame',
    '@collid': '13824215',
    'name': {'@sortindex': '1', '#text': '10 Days in Europe'},
    'yearpublished': '2002',
    'image': 'https://cf.geekdo-images.com/7jXx06W2pr66YKWrZ7Dywg__original/img/ylUIeQ-GifItuTGgLWGWhyA1cus=/0x0/filters:format(png)/pic4895073.png',
    'thumbnail': 'https://cf.geekdo-images.com/7jXx06W2pr66YKWrZ7Dywg__thumb/img/eKUhAbBWedJv_RIsUA47BpQSTQw=/fit-in/200x150/filters:strip_icc()/pic4895073.png',
    'stats': {'@minplayers': '2',
     '@maxplayers': '4',
     '@minplaytime': '20',
     '@maxplaytime': '30',
     '@playingtime': '30',
     '@numowned': '2576',
     'rating': {'@value': '7.5',
      'usersrated': {'@value': '2062'},
      'average': {'@value': '6.60089'},
      'bayesaverage': {'@value': '6.0562'},
      'std

In [160]:
for game in data_dict['items']['item']:
    print(game['stats']['rating']['@value'])

7.5
7.5
5
6
1
7
8
6
6.7
7.8
1.5
8.8
4
7.5
6
6
6.5
7.5
4
6.2
5.1
5
7.2
5.8
7
5
7
7
5
6.8
3
7
3
8
8.8
8
5.8
7.6
7.7
7.1
7
5
7
6
6.9
8
2
4
9
9
5
7
7.3
7.5
7
7.5
7.5
3
6.8
5
5
10
3
7
8
6
7.5
6
6
7
9.8
7
8.5
8
7
6.8
6.5
7.5
5
6
10
9
6
2.5
4.5
6.5
6.9
5
6.6
5
5.5
4.5
5.5
8
6.7
7
7.2
7.5
8
7.2
7.5
5
6
8
8
7.6
8
3.7
6
8
9.5
8
6
9
10
5
4
6
9.2
5
7
6.5
6.8
6.8
8
7.5
7
6.5
6.4
7.8
6.9
3
4
7
7
7
6
6
4
7
6.5
6.5
3
4
7
6
7
7.5
8
8.5
6.7
5.6
8.5
9
7.5
7.7
8
4.7
7
5.2
6.7
7
5.8
4.5
7
9
6.8
3
3.6
8
9
6
7
6.8
2
3
2
1.5
3
8
2.4
3.8
5.6
8.5
7.5
3.1
6.2
2
7
7
7
6
5
3.5
6.5
5
5.2
7.5
6.7
6.66
1
1
5.1
5
5
5
4
8
6
6
4.8
7
8.8
7.2
9
7.1
7
5.5
6.5
8.2
4
8.5
6.5
6.5
1
2.5
7.5
8.8
1.2
8
2
2
7.6
5.8
7.5
7
5.8
7
3
7.2
5
3
7
5.5
6
4
8
5.5
4
6.7
3
6.5
6.5
6.5
7
8
6.5
7
6.8
4
6.5
6
9
9
9
8.8
2.5
2
4.8
3.5
5.8
5
5.5
6.5
7
6.5
8
8
5.5
8
2.2
2
8
4.5
3
6.6
6
7
6
7
3
4.5
5
7.8
6
6.8
6.5
6.5
5.5
7.8
6.2
8
8
9
7
2
8.5
1.5
5.2
6
2.5
8.5
6
9
8.8
8.8
8
8.2
2.5
8
6
7
5
7.4
2
6
5
7
8
8
8.5
10
9
8.8
7
9.9
8.8
8
9.5
9.9
8
10
9.5
9.

In [136]:
data_dict['items']['item'][2]

{'@objecttype': 'thing',
 '@objectid': '7865',
 '@subtype': 'boardgame',
 '@collid': '4714477',
 'name': {'@sortindex': '1', '#text': '10 Days in Africa'},
 'yearpublished': '2003',
 'image': 'https://cf.geekdo-images.com/B1Bji6N6r1kUvI_Glp7U1w__original/img/3Y_svR8FUmL-yicbCq3bwBvSYKc=/0x0/filters:format(jpeg)/pic1229634.jpg',
 'thumbnail': 'https://cf.geekdo-images.com/B1Bji6N6r1kUvI_Glp7U1w__thumb/img/ZRfdBTjVmigOLsZxf_dh-iCXRuM=/fit-in/200x150/filters:strip_icc()/pic1229634.jpg',
 'stats': {'@minplayers': '2',
  '@maxplayers': '4',
  '@minplaytime': '20',
  '@maxplaytime': '30',
  '@playingtime': '30',
  '@numowned': '2265',
  'rating': {'@value': '7',
   'usersrated': {'@value': '1988'},
   'average': {'@value': '6.56097'},
   'bayesaverage': {'@value': '6.01878'},
   'stddev': {'@value': '1.18885'},
   'median': {'@value': '0'}}},
 'status': {'@own': '0',
  '@prevowned': '0',
  '@fortrade': '0',
  '@want': '0',
  '@wanttoplay': '0',
  '@wanttobuy': '0',
  '@wishlist': '0',
  '@pr

In [123]:
data_dict['message']#['items']['@totalitems']

'Your request for this collection has been accepted and will be processed.  Please try again later for access.'

In [119]:
data_dict['items']['item'][4].keys()

dict_keys(['@objecttype', '@objectid', '@subtype', '@collid', 'name', 'yearpublished', 'image', 'thumbnail', 'stats', 'status', 'numplays', 'comment'])

In [85]:
data_dict['items']['item'][0]['@objectid']

'2498'

In [86]:
data_dict['items']['item'][0]['@subtype']

'boardgame'

In [94]:
data_dict['items']['item'][2]['comment']

'I can just see [person=9]Moon[/person] and [person=235]Weissblum[/person] getting up from a pleasant game of [thing=917]Rack-O[/thing], saying, "That\'s such a nice core mechanism for a game.  How can we make it more interesting?"'

In [118]:
data_dict['items']['item'][4]['status']

{'@own': '0',
 '@prevowned': '1',
 '@fortrade': '0',
 '@want': '0',
 '@wanttoplay': '0',
 '@wanttobuy': '0',
 '@wishlist': '0',
 '@preordered': '0',
 '@lastmodified': '2010-05-26 09:51:24'}